In [15]:
#Imports
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import hvplot.pandas
import plotly.figure_factory as ff
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.dask
from sklearn.preprocessing import LabelEncoder

In [2]:
#File Path
file_path= "crypto_data.csv"
crypto_df=pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [5]:
#Remove all cryptocurrencies that aren’t trading
def change_string(IsTrading):
    if IsTrading == "False":
        return 0
    else:
        return 1
    
crypto_df["IsTrading"] = crypto_df["IsTrading"].apply(change_string)
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [6]:
#Remove all cryptocurrencies that don’t have an algorithm defined
crypto_df = crypto_df[crypto_df['Algorithm'].isna() == False]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [7]:
#Remove all cryptocurrencies with at least one null value
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [8]:
#Remove all cryptocurrencies without coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] != 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [10]:
#Store the names of all cryptocurrencies on a DataFramed named coins_name, and use the crypto_df.index as the index for this new DataFrame.
coins_name=crypto_df.filter(["CoinName"], axis=1)
coins_name.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [11]:
#Remove the CoinName column
crypto_df.drop(columns=["CoinName"], axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [19]:
#Create dummies variables for all of the text features, and store the resulting data on a DataFrame named X.
le = LabelEncoder()
X = crypto_df
X['Algorithm'] = le.fit_transform(X['Algorithm'])
X['ProofType'] = le.fit_transform(X['ProofType'])

X

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,53,16,4.199995e+01,42
404,53,16,1.055185e+09,532000000
1337,67,16,2.927942e+10,314159265359
BTC,47,13,1.792718e+07,21000000
ETH,20,13,1.076842e+08,0
...,...,...,...,...
ZEPH,47,2,2.000000e+09,2000000000
GAP,53,16,1.493105e+07,250000000
BDX,10,13,9.802226e+08,1400222610
ZEN,18,13,7.296538e+06,21000000


In [20]:
#Use the StandardScaler from sklearn (Links to an external site.) to standardize all of the data from the X DataFrame. Remember, this is important prior to using PCA and K-means algorithms
X_scaled=StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[ 0.39335561  0.8871567  -0.11674788 -0.15286468]
 [ 0.39335561  0.8871567  -0.09358885 -0.14499604]
 [ 1.21439604  0.8871567   0.52587231  4.4937636 ]
 [ 0.04148114  0.00878917 -0.11635442 -0.15255408]
 [-1.54195398  0.00878917 -0.11438445 -0.15286468]]


In [22]:
pca = PCA(n_components=3)

In [23]:
crypto_pca = pca.fit_transform(X_scaled)

In [25]:
pcs_df=pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2","PC 3"], index=crypto_df.index
)

pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.412665,0.823150,0.357227
404,-0.391306,0.827873,0.358336
1337,3.143152,2.186748,0.487273
BTC,-0.190301,-0.003829,-0.043688
ETH,-0.051013,-1.161387,1.020473
LTC,-0.219679,0.253752,-0.280044
DASH,-0.473703,1.337814,-0.115708
XMR,-0.011450,-1.504754,1.335799
ETC,-0.048798,-1.160925,1.020929
ZEC,-0.042085,-1.247450,1.099354


In [26]:
#Create an elbow curve to find the best value for K, and use the pcs_df DataFrame
inertia = []
k = list(range(1, 11))

for i in k:
	km = KMeans(n_clusters=i, random_state=0)
	km.fit(pcs_df)
	inertia.append(km.inertia_)
    
elbow_crypto = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_crypto)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [27]:
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add the predicted class columns
pcs_df["class"] = model.labels_

pcs_df

,PC 1,PC 2,PC 3,class
42,-0.412665,0.823150,0.357227,1
404,-0.391306,0.827873,0.358336,1
1337,3.143152,2.186748,0.487273,1
BTC,-0.190301,-0.003829,-0.043688,1
ETH,-0.051013,-1.161387,1.020473,0
...,...,...,...,...
ZEPH,0.560750,-2.082122,-2.376795,2
GAP,-0.409905,0.823729,0.357770,1
BDX,0.027548,-1.584277,1.417630,0
ZEN,-0.042086,-1.247450,1.099354,0


In [28]:
clustered_df = coins_name.join(pcs_df)

clustered_df

,CoinName,PC 1,PC 2,PC 3,class
42,42 Coin,-0.412665,0.823150,0.357227,1
404,404Coin,-0.391306,0.827873,0.358336,1
1337,EliteCoin,3.143152,2.186748,0.487273,1
BTC,Bitcoin,-0.190301,-0.003829,-0.043688,1
ETH,Ethereum,-0.051013,-1.161387,1.020473,0
...,...,...,...,...,...
ZEPH,ZEPHYR,0.560750,-2.082122,-2.376795,2
GAP,Gapcoin,-0.409905,0.823729,0.357770,1
BDX,Beldex,0.027548,-1.584277,1.417630,0
ZEN,Horizen,-0.042086,-1.247450,1.099354,0


In [37]:
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

ValueError: Value of 'hover_data_0' is not the name of a column in 'data_frame'. Expected one of ['CoinName', 'PC 1', 'PC 2', 'PC 3', 'class'] but received: Algorithm

In [39]:
clustered_df.hvplot.table(
    columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'],
    width =500
)

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

In [40]:
clustered_df.hvplot.scatter(
	x="TotalCoinsMined",
	y="TotalCoinSupply",
	hover_cols=["CoinName"],
	by="class",
)

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['TotalCoinsMined', 'TotalCoinSupply']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html